In [126]:
import requests
import re
from bs4 import BeautifulSoup
import string
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from pyshadow.main import Shadow
import pandas as pd
import csv, os
from selenium.common.exceptions import NoSuchElementException

In [127]:
# define Python user-defined exceptions
class NoCitation(Exception):
    pass

In [128]:
col=['AU1_r','AU2_r','AU3_r','AU4_r','AU5_r','TI_r','TY']
for i in range(1,11):
    col.append('AU'+str(i))
col = col + ["TI","JO","VL","IS","SN","DO","SP","EP"]

In [129]:
def Download_Citation(Volume, Issue):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #driver.execute_script("window.open('https://onlinelibrary.wiley.com/toc/1534875x/"+str(Volume)+"/"+str(Volume)+"/"+str(Issue)+"/');")
    driver.get("https://onlinelibrary.wiley.com/toc/1534875x/"+str(Volume)+"/"+str(Volume)+"/"+str(Issue))
    driver.find_element(By.CSS_SELECTOR, "a[class='bulk-download-option'] span").click()
    Research_Articles = driver.find_elements(By.CLASS_NAME,'checkbox--primary')
    for a in Research_Articles[1:]:
        a.click()
    driver.find_element(By.CSS_SELECTOR, ".bulk-wizard.exportCitationsForm").click()
    driver.find_element(By.CSS_SELECTOR, ".export-citations__submit").click()
    driver.quit()

In [130]:
def Find_Authors(doi):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #driver.execute_script("window.open(\'"+str(doi)+"\');")
    driver.get(doi)
    line = []
    try:
        a = driver.find_element(By.CSS_SELECTOR, "div[role='button']")
    except NoSuchElementException:
        line =["","","","",""]
        raise NoCitation
    else:
        driver.find_element(By.CSS_SELECTOR, "div[role='button']").click()
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='article__body ']//li[2]"))
        )
    finally:
        citing_lit= driver.find_elements(By.XPATH, "//div[@class='article__body ']//li[2]")
    #authors = driver.find_element(By.CSS_SELECTOR, ".rlist.cited-by__list")
    for a in citing_lit:
        authors = a.find_elements(By.CLASS_NAME,'hlFld-ContribAuthor')
    AU=[]
    for a in authors:
        AU.append(a.text[:-1])
    line = line + AU
    while(len(line)<5):
        line.append("")
    while(len(line)>5):
        line.pop()
    TI = driver.find_element(By.CSS_SELECTOR,'body > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > main:nth-child(1) > div:nth-child(1) > div:nth-child(1) > section:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > article:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(5) > div:nth-child(3) > section:nth-child(1) > div:nth-child(1) > div:nth-child(2) > ul:nth-child(3) > li:nth-child(2) > span:nth-child(2)')
    line.append(TI.text)
    driver.quit()
    return line

In [131]:
def Read_Citation():
    file = open('/Users/langchen777/Downloads/pericles_exported_citations.txt', 'r')
    df = pd.DataFrame(columns=col)
    Lines = file.readlines()
    count = 0
    for line in Lines:
        start = line[:2]
        if start == "TY":
            count = count + 1
            row = []
            row.append(line.split("-",1)[1].strip())
        elif start == "AU":
            row.append(line.split("-",1)[1].strip())
        elif start == "TI":
            while(len(row)<11):
                row.append("")
            while(len(row)>11):
                row.pop()
            row.append(line.split("-",1)[1].strip())
        elif start in ["JO","VL","IS","SN","DO","SP","EP"]:
            row.append(line.split("-",1)[1].strip())
            if start in "DO":
                doi = line.split("-",1)[1].strip()
        elif start == "ER":
        combine = row + Find_Authors(doi)
        a_series = pd.Series(combine, index = col)
        df = df.append(a_series, ignore_index=True)
    os.remove("/Users/langchen777/Downloads/pericles_exported_citations.txt")
    return df

In [132]:
def Return_Issue(i):
    if i in range(1,5):
        volume = 1978
    else:
        volume = 1979 + (i-1)//4
    Download_Citation(volume, i)
    df=Read_Citation()
    return df

In [133]:
df = Return_Issue(84)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/langchen777/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
<ipython-input-129-c8249f198d68>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/langchen777/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/langchen777/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver

In [134]:
df

,AU1_r,AU2_r,AU3_r,AU4_r,AU5_r,TI_r,TY,AU1,AU2,AU3,...,AU9,AU10,TI,JO,VL,IS,SN,DO,SP,EP
0,JOUR,"Watt, James H.",,,,,,,,,...,1097-6736,https://doi.org/10.1002/ev.1151,23,43,Sergio Mauceri,Maria Paola Faggiano,Luca Di Censi,,,Online Research Practice and Integrated Perspe...
1,JOUR,"Watt, James H.",,,,,,,,,...,1097-6736,https://doi.org/10.1002/ev.1151,23,43,Sergio Mauceri,Maria Paola Faggiano,Luca Di Censi,,,Online Research Practice and Integrated Perspe...
